In [ ]:
# 3
# transpose workbook
import os
import openpyxl
import copy

In [ ]:
# get file path
while True:
    file_path = input('\nEnter the path of the workbook to be transposed (.xlsx extension is optional)\n> ')
    if not file_path.endswith('.xlsx'):
        file_path += '.xlsx'
        
    file_path = os.path.abspath(file_path)
    if os.path.exists(file_path):
        break
    print('\nFile not found')

# access workbooks
source_workbook = openpyxl.load_workbook(file_path, data_only=True)
target_workbook = openpyxl.Workbook()

# loop through sheets
for sheetname in copy.copy(target_workbook.sheetnames):
    del target_workbook[sheetname]

for sheetname in source_workbook.sheetnames:
    source_sheet = source_workbook[sheetname]
    target_sheet = target_workbook.create_sheet(sheetname)

    if source_sheet.freeze_panes is not None:
        # transpose freeze panes
        target_sheet.freeze_panes = target_sheet.cell(column=source_sheet.freeze_panes.row, row=source_sheet.freeze_panes.column)

    # transpose rows
    for source_row in list(source_sheet.rows):
        for source_cell in source_row:
            target_cell = target_sheet.cell(row=source_cell.column, column=source_cell.row, value=source_cell.value)

            # copy styles
            if source_cell.has_style:
                target_cell.number_format = copy.copy(source_cell.number_format)
                target_cell.font = copy.copy(source_cell.font)
                target_cell.fill = copy.copy(source_cell.fill)
                target_cell.border = copy.copy(source_cell.border)
                target_cell.alignment = copy.copy(source_cell.alignment)
                target_cell.protection = copy.copy(source_cell.protection)

# save new workbook
new_file_name = 'transposed ' + os.path.basename(file_path)
new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)
target_workbook.save(new_file_path)



# close workbooks
source_workbook.close()
target_workbook.close()

In [ ]:
# 4
# Excel from texts
import os
import sys
import pyinputplus as pyip
import openpyxl

In [ ]:
# get directory of text files
while True:
    folder_path = input('Enter the path of the directory containing your text files.\n> ')
    folder_path = os.path.abspath(folder_path)
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        break
    print(f'\nThe directory {folder_path} cannot be found.')
    
# get list of text file dicts (name and path)
text_files = [{'name' : file_name, 'path' : os.path.join(folder_path, file_name)} for file_name in os.listdir(folder_path) if file_name.endswith('.txt')]

if len(text_files) == 0:
    print('\nNo text files were found.')
    sys.exit()

print('\nThe following text files were found:')
for i in range(len(text_files)):
    print('\t-> ' + text_files[i]['name'])

# read lines to dict
for text_file in text_files:
    with open(text_file['path'], 'r') as file:
        text_file['lines'] = [line.strip() for line in file.readlines()]

# rows or columns
row_column = pyip.inputChoice(['r', 'c', 'q'], '\nDoes each text file correspond to a (r)ow or a (c)olumn? Enter (q) to quit.\n> ')
if row_column == 'q':
    sys.exit()

# initialize workbook
workbook = openpyxl.Workbook()
sheet = workbook.active

# loop to populate workbook
if row_column == 'r':
    for i, text_file in enumerate(text_files):
        for j, line in enumerate(text_file['lines']):
            sheet.cell(row=i + 1, column=j + 1, value = line)
elif row_column =='c':
    for i, text_file in enumerate(text_files):
        for j, line in enumerate(text_file['lines']):
            sheet.cell(row=j + 1, column=i + 1, value = line)

# save workbook
workbook_path = os.path.join(folder_path, os.path.basename(folder_path) + '.xlsx')
print(f'\nSaving workbook to {workbook_path}')
workbook.save(workbook_path)

#close workbook
workbook.close()

print('\nDone.')

In [1]:
# 5
# texts from Excel
import os
import sys
import pyinputplus as pyip
import openpyxl

In [8]:
# get file path
while True:
    file_path = input('\nEnter the path of the workbook (.xlsx extension is optional)\n> ')
    if not file_path.endswith('.xlsx'):
        file_path += '.xlsx'
        
    file_path = os.path.abspath(file_path)
    if os.path.exists(file_path):
        break
    print('\nFile not found')

# create directory
text_directory_path = os.path.join(os.path.dirname(file_path), os.path.basename(file_path)[:-5])
if not os.path.exists(text_directory_path):
    print(f'\nCreating directory {text_directory_path}')
    os.mkdir(text_directory_path)
    
# rows or columns
row_column = pyip.inputChoice(['r', 'c', 'q'], '\nShould each text file correspond to a (r)ow or a (c)olumn? Enter (q) to quit.\n> ')
if row_column == 'q':
    sys.exit()
    
# open workbook
workbook = openpyxl.load_workbook(file_path, data_only=True)
sheet = workbook.active

# loop over workbook to create a dict of line lists
text_dict = {}
if row_column == 'r':
    for row in list(sheet.rows):
        for cell in row:
            key = 'row' + str(cell.row)
            text_dict.setdefault(key, [])
            text_dict[key].append(cell.value + '\n')
elif row_column == 'c':
    for column in list(sheet.columns):
        for cell in column:
            key = 'col' + cell.column_letter
            text_dict.setdefault(key, [])
            text_dict[key].append(cell.value + '\n')

# close workbook
workbook.close()

# loop over dict to create text files
for key in text_dict:
    text_file_path = os.path.join(text_directory_path, key + '.txt')
    print(f'\nWriting to {text_file_path}')
    with open(text_file_path, 'w') as text_file:
        text_file.writelines(text_dict[key])

print('\nDone.')


Enter the path of the workbook (.xlsx extension is optional)
> text files\text files

Should each text file correspond to a (r)ow or a (c)olumn? Enter (q) to quit.
> c

Writing to C:\Users\jacob\Documents\CompSci\PythonProjects\Automate the Boring Stuff\Working with Excel Spreadsheets\text files\text files\colA.txt

Writing to C:\Users\jacob\Documents\CompSci\PythonProjects\Automate the Boring Stuff\Working with Excel Spreadsheets\text files\text files\colB.txt

Done.
